# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Mod 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging than the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

**You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`** For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

> Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

> * The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
* **The team's win percentage on days where it was raining during games in the 2011 season.**

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, **so go ahead and just use the weather in Berlin, Germany as a proxy for this information.** If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well-structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at a minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

___

# STUDY GROUP

In [1]:
!pip install -U fsds_100719
from fsds_100719.imports import *
import sqlite3

fsds_1007219  v0.5.9 loaded.  Read the docs: https://fsds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds_100719,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


In [6]:
import json,os,sys

with open('/Users/jamesirving/.secret/dark_sky.json') as f:
    api_key = json.loads(f.read())
api_key.keys()

dict_keys(['secret key'])

In [7]:
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [8]:
cur.execute("select * from matches where season=2011")
cur.description

(('Match_ID', None, None, None, None, None, None),
 ('Div', None, None, None, None, None, None),
 ('Season', None, None, None, None, None, None),
 ('Date', None, None, None, None, None, None),
 ('HomeTeam', None, None, None, None, None, None),
 ('AwayTeam', None, None, None, None, None, None),
 ('FTHG', None, None, None, None, None, None),
 ('FTAG', None, None, None, None, None, None),
 ('FTR', None, None, None, None, None, None))

In [9]:
col_names = [x[0] for x in cur.description]
col_names

['Match_ID',
 'Div',
 'Season',
 'Date',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR']

matches = pd.DataFrame(cur.fetchall(),columns=col_names)
matches

In [11]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 9 columns):
Match_ID    992 non-null int64
Div         992 non-null object
Season      992 non-null int64
Date        992 non-null object
HomeTeam    992 non-null object
AwayTeam    992 non-null object
FTHG        992 non-null int64
FTAG        992 non-null int64
FTR         992 non-null object
dtypes: int64(4), object(5)
memory usage: 69.9+ KB


In [14]:
df = matches.loc[matches['Season'] == 2011]
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
...,...,...,...,...,...,...,...,...,...
987,44870,E0,2011,2012-05-13,Sunderland,Man United,0,1,A
988,44871,E0,2011,2012-05-13,Swansea,Liverpool,1,0,H
989,44872,E0,2011,2012-05-13,Tottenham,Fulham,2,0,H
990,44873,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A


In [15]:
test_date = df['Date'][0]
test_date

'2012-03-31'

In [27]:
import requests
def api_request(time,verbose=1):
    # request_url = "https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]"
    lat =52.5200
    long= 13.4050

    time_suffix = "T12:00:00"
    check_date = time+time_suffix

    template_url= f"https://api.darksky.net/forecast/{api_key['secret key']}/{lat},{long},{check_date}"
    if verbose==2:
        print(template_url)
    
        
    
    response = requests.get(template_url )
    
    if response.status_code == 200:
        if verbose>0: 
            print('Success')
    else: 
        print(f"Error: status code = {response.status_code}")
    
    return response

In [29]:
resp = api_request(test_date,verbose=2)

https://api.darksky.net/forecast/a94b3dbf787759eb6354a8da1e93fdb4/52.52,13.405,2012-03-31T12:00:00
Success


In [34]:
r_dict = resp.json()
r_dict.keys()

dict_keys(['latitude', 'longitude', 'timezone', 'currently', 'hourly', 'daily', 'flags', 'offset'])

In [ ]:
def get_weather_from_response(response):
    r_dict = resp.json()
    weather = r_dict['daily']['data'][0]['icon']


In [41]:
# fs.ds.dict_dropdown(r_dict)
r_dict['daily']['data'][0]['icon']


'rain'

In [74]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    
    def __init__(self,api_key, lat =52.5200, long= 13.4050,):
        """Sets the lat/long for the location, and stores api_key."""
        self.lat= lat
        self.long=long
        self.base_url  = "https://api.darksky.net/forecast/"
        self.api_key = api_key
        
    
    def get_weather_for_date(self, date,verbose=1):
        """Checks dark_sky api for weather from date. """
        import requests
        # request_url = "https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]"

        time_suffix = "T12:00:00"
        check_date = date+time_suffix

        template_url= f"https://api.darksky.net/forecast/{self.api_key}/{self.lat},{self.long},{check_date}"
        if verbose==2:
            print(template_url)

        ## Get and check response
        response = requests.get(template_url )
        if response.status_code == 200:
            
            if verbose>0: 
                print('Success')
        else: 
            print(f"Error: status code = {response.status_code}")
            
        # call helper method
        return self._get_weather_from_response(response)
    
    def _get_weather_from_response(self,response):
        """Extracts weather icon from dark_sky api response"""
        r_dict = resp.json()
        return r_dict['daily']['data'][0]['icon']
        
    def get_weather_for_date_list(self,dates):    
        import time
        weather_list = []
        
        for date in dates:
            time.sleep(0.5)
            weather_list.append(self.get_weather_for_date(date))
            
            
#         self._weather_list     
        return weather_list



### Examining the Dataset

> Table: Matches
Match_ID (int): unique ID per match

Div (str): identifies the division the match 
was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)

Season (int): Season the match took place in (usually covering the period of August till May of the following year)

Date (str): Date of the match

HomeTeam (str): Name of the home team

AwayTeam (str): Name of the away team

FTHG (int) (Full Time Home Goals): Number of goals scored by the home team

FTAG (int) (Full Time Away Goals): Number of goals scored by the away team

FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

In [53]:
ht = list(df['HomeTeam'].unique())
at = list(df['AwayTeam'].unique())
len(ht)==len(at)

## Make list of teams
ht.extend(at)
team_list = list(set(ht))
team_list

True

In [83]:
class Team():
    """
    > * The name of the team
    * The total number of goals scored by the team during the 2011 season
    * The total number of wins the team earned during the 2011 season
    * A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
    * **The team's win percentage on days where it was raining during games in the 2011 season.**
    """
    def __init__(self, name, total_goals_2011, total_wins_2011):
        self.name= name
        self.total_goals_2011 = total_goals_2011
        self.total_wins_2011 = total_wins_2011
        
#     def calc_stat(self):
#         pass
    
#         histogram = None
#         percent_rainy_wins = None
# ):

    #     def __init__(self,n]

# df
        

In [47]:
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
...,...,...,...,...,...,...,...,...,...
987,44870,E0,2011,2012-05-13,Sunderland,Man United,0,1,A
988,44871,E0,2011,2012-05-13,Swansea,Liverpool,1,0,H
989,44872,E0,2011,2012-05-13,Tottenham,Fulham,2,0,H
990,44873,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A


In [61]:
df_team = df.groupby('HomeTeam').get_group(team_list[0])
val_counts = df_team['FTR'].value_counts()
val_counts['H']
# type(val_counts)



11

In [62]:
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
...,...,...,...,...,...,...,...,...,...
987,44870,E0,2011,2012-05-13,Sunderland,Man United,0,1,A
988,44871,E0,2011,2012-05-13,Swansea,Liverpool,1,0,H
989,44872,E0,2011,2012-05-13,Tottenham,Fulham,2,0,H
990,44873,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A


In [63]:
df["Date"].nunique()

165

In [67]:
# wget=WeatherGetter(api_key=api_key['secret key'])
# wget.get_weather_for_

In [44]:
    ou
    
tot_goals 
tot_wins

'2012-03-31'

In [75]:
dates_to_check = list(df['Date'].unique())[:20]
dates_to_check

['2012-03-31',
 '2011-12-11',
 '2011-08-13',
 '2011-11-27',
 '2012-02-18',
 '2012-01-20',
 '2012-02-04',
 '2012-04-21',
 '2011-09-18',
 '2011-10-23',
 '2011-10-01',
 '2012-03-03',
 '2011-08-27',
 '2012-03-17',
 '2011-11-06',
 '2012-05-05',
 '2012-04-11',
 '2011-12-17',
 '2012-02-03',
 '2011-10-29']

In [76]:
wget = WeatherGetter(api_key['secret key'])
weather_results = wget.get_weather_for_date_list(dates_to_check)

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


In [84]:
weather_dict = dict(zip(dates_to_check,weather_results))
weather_dict

{'2012-03-31': 'rain',
 '2011-12-11': 'rain',
 '2011-08-13': 'rain',
 '2011-11-27': 'rain',
 '2012-02-18': 'rain',
 '2012-01-20': 'rain',
 '2012-02-04': 'rain',
 '2012-04-21': 'rain',
 '2011-09-18': 'rain',
 '2011-10-23': 'rain',
 '2011-10-01': 'rain',
 '2012-03-03': 'rain',
 '2011-08-27': 'rain',
 '2012-03-17': 'rain',
 '2011-11-06': 'rain',
 '2012-05-05': 'rain',
 '2012-04-11': 'rain',
 '2011-12-17': 'rain',
 '2012-02-03': 'rain',
 '2011-10-29': 'rain'}

In [79]:
df['Use Me'] = df['Date'].isin(dates_to_check)
df=df.loc[df['Use Me'] ==True ]
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Use Me
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,True
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,True
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,True
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,True
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,True
...,...,...,...,...,...,...,...,...,...,...
950,44833,E0,2011,2012-04-21,Bolton,Swansea,1,1,D,True
951,44834,E0,2011,2012-04-21,Fulham,Wigan,2,1,H,True
952,44835,E0,2011,2012-04-21,Newcastle,Stoke,3,0,H,True
953,44836,E0,2011,2012-04-21,QPR,Tottenham,1,0,H,True


In [85]:
df['weather'] = df['Date'].apply(lambda x: weather_dict[x])
df

//anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Use Me,weather
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,True,rain
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,True,rain
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,True,rain
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,True,rain
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,True,rain
...,...,...,...,...,...,...,...,...,...,...,...
950,44833,E0,2011,2012-04-21,Bolton,Swansea,1,1,D,True,rain
951,44834,E0,2011,2012-04-21,Fulham,Wigan,2,1,H,True,rain
952,44835,E0,2011,2012-04-21,Newcastle,Stoke,3,0,H,True,rain
953,44836,E0,2011,2012-04-21,QPR,Tottenham,1,0,H,True,rain


In [82]:
cur_team = Team()
cur_team.

In [80]:
def make_team(df,):
    
    
    ht = list(df['HomeTeam'].unique())
    at = list(df['AwayTeam'].unique())
    len(ht)==len(at)

    ## Make list of teams
    ht.extend(at)
    team_list = list(set(ht))
    
    team_dict={}
    for team in team_list:

        cur_team = Team()
        

        total_wins = 0
        total_goals = 0

        # get homegames 
        df_team = df.groupby('HomeTeam').get_group(team)
        


        val_counts = df_team['FTR'].value_counts()
        total_wins+=val_counts['H'] 
        total_goals +=df_team['FTHG'].sum()

        # get away ames 
        df_team = df.groupby('Awayteam').get_group(team)
        val_counts = df_team['FTR'].value_counts()
        total_wins+=val_counts['A']
        total_goals +=df_team['FTAG'].sum()


        #


20

In [ ]:
class MongoHandler():
    pass

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!